# Analyzing weather data

## Set up environment/get data

### Import Python modules and variables

In [3]:
# Add the project root to sys.path
import sys
import os
sys.path.append(os.path.abspath(".."))

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from IPython.display import HTML
from projectconfig.config import raw_data_path

### Import data

In [5]:
weather = pd.read_csv(raw_data_path, parse_dates=['Date_Time'])
weather.head()

,Location,Date_Time,Temperature_C,Humidity_pct,Precipitation_mm,Wind_Speed_kmh
0,San Diego,2024-01-14 21:12:46,10.683001,41.195754,4.020119,8.233540
1,San Diego,2024-05-17 15:22:10,8.734140,58.319107,9.111623,27.715161
2,San Diego,2024-05-11 09:30:59,11.632436,38.820175,4.607511,28.732951
3,Philadelphia,2024-02-26 17:32:39,-8.628976,54.074474,3.183720,26.367303
4,San Antonio,2024-04-29 13:23:51,39.808213,72.899908,9.598282,29.898622
